### Imports

In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

In [16]:
import pytz

### Config

In [2]:
PG_HOST="localhost"
PG_DATABASE="mr_alitas"
PG_USER="postgres"
PG_PASS="000111"

try:
  alchemyEngine = create_engine('postgresql+psycopg2://'+PG_USER+':'+PG_PASS+'@'+PG_HOST+':5432/'+PG_DATABASE+'', pool_recycle=3600)
except Exception as e:
  print("Error al iniciar postgresql: " + str(e))

In [6]:
PG_HOST='monorail.proxy.rlwy.net'
PG_DATABASE='railway'
PG_USER='postgres'
PG_PASS='4EAFBd5-5da2D-6gd4EGg2f1G2B32df3'
PG_PORT=15770

try:
  alchemyEngine = create_engine('postgresql+psycopg2://'+PG_USER+':'+PG_PASS+'@'+PG_HOST+':'+str(PG_PORT)+'/'+PG_DATABASE+'', pool_recycle=3600)
except Exception as e:
  print("Error al iniciar postgresql: " + str(e))

### Funciones

In [4]:
def pg_connect():
  connection = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASS,
    port=PG_PORT
  )
  return connection

### DATA ANALYSIS BY USER

In [7]:
with alchemyEngine.connect() as dbConnection:
  VENTAS = pd.read_sql(
    "SELECT * FROM ventas where id_usuario_fk = %s",
    dbConnection,
    params=(1,)
  )

In [8]:
VENTAS

,id_venta,num_productos,total,valor_pagado,cambio,id_usuario_fk,fecha_creacion


In [7]:
VENTAS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_venta        18 non-null     int64         
 1   num_productos   18 non-null     int64         
 2   total           18 non-null     float64       
 3   valor_pagado    18 non-null     float64       
 4   cambio          18 non-null     float64       
 5   fecha_creacion  18 non-null     datetime64[ns]
 6   id_usuario_fk   18 non-null     int64         
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 1.1 KB


In [13]:
current_year = VENTAS['fecha_creacion'].dt.year.max()
current_month = VENTAS['fecha_creacion'].dt.month.max()
current_month_data = VENTAS[(VENTAS['fecha_creacion'].dt.year == current_year) & (VENTAS['fecha_creacion'].dt.month == current_month)]
current_month_sales = current_month_data.groupby(current_month_data['fecha_creacion'].dt.date)['total'].sum().reset_index()


In [9]:
current_month_data

,id_venta,num_productos,total,valor_pagado,cambio,fecha_creacion,id_usuario_fk
0,1,3,6.0,10.0,4.0,2023-12-22 23:40:29.807236,1
1,2,3,6.0,10.0,4.0,2023-12-22 23:50:33.534514,1
2,3,3,6.0,10.0,4.0,2023-12-22 23:51:42.694241,1
3,4,1,7.5,10.0,2.5,2023-12-22 23:52:49.640618,1
4,5,2,4.0,5.0,1.0,2023-12-22 23:54:17.137453,1
5,6,1,2.0,5.0,3.0,2023-12-22 23:55:10.123227,1
6,7,1,2.0,5.0,3.0,2023-12-22 23:58:41.016204,1
7,8,2,4.0,5.0,1.0,2023-12-22 23:59:32.086504,1
8,9,1,2.0,2.0,0.0,2023-12-23 00:00:47.269447,1
9,10,1,2.0,2.0,0.0,2023-12-23 00:01:59.489409,1


In [19]:
current_month_data.groupby(current_month_data['fecha_creacion'].dt.year)['total'].sum().reset_index()

,fecha_creacion,total
0,2023,132.0


In [14]:
current_month_sales

,fecha_creacion,total
0,2023-12-22,37.5
1,2023-12-23,14.0
2,2023-12-27,80.5


In [24]:
dates_list = [date.strftime("%d-%m-%Y") for date in current_month_sales['fecha_creacion'].to_list()]
dates_list

['22-12-2023', '23-12-2023', '27-12-2023']

In [26]:
sales = current_month_sales['total'].tolist()
sales

[37.5, 14.0, 80.5]

### DATA ANALYSIS GENERAL

In [27]:
with alchemyEngine.connect() as dbConnection:
  VENTAS_GENERAL = pd.read_sql(
    "SELECT * FROM ventas v left join usuarios u on u.id_usuario = v.id_usuario_fk ",
    dbConnection,
  )

In [29]:
VENTAS_GENERAL.head(2)

,id_venta,num_productos,total,valor_pagado,cambio,fecha_creacion,id_usuario_fk,id_usuario,username,name_user
0,1,3,6.0,10.0,4.0,2023-12-22 23:40:29.807236,1,1,ximena,Ximena García
1,2,3,6.0,10.0,4.0,2023-12-22 23:50:33.534514,1,1,ximena,Ximena García


In [35]:
monthly_user_sales = VENTAS_GENERAL.groupby([VENTAS_GENERAL['fecha_creacion'].dt.to_period('D'), 'name_user'])['total'].sum().reset_index()

monthly_user_sales

,fecha_creacion,name_user,total
0,2023-12-22,Ximena García,37.50
1,2023-12-23,Alexandra Sinche,69.35
2,2023-12-23,Ximena García,14.00
3,2023-12-27,Ximena García,87.50


In [36]:
monthly_user_sales['fecha_creacion'].tolist()

[Period('2023-12-22', 'D'),
 Period('2023-12-23', 'D'),
 Period('2023-12-23', 'D'),
 Period('2023-12-27', 'D')]

In [38]:
monthly_user_sales = VENTAS_GENERAL.groupby([VENTAS_GENERAL['fecha_creacion'].dt.to_period('M'), 'name_user'])['total'].sum().reset_index()
monthly_user_sales['fecha_creacion'] = monthly_user_sales['fecha_creacion'].dt.strftime("%m-%Y")
monthly_user_sales['fecha_creacion'].tolist()

['12-2023', '12-2023']

In [40]:
monthly_user_sales = VENTAS_GENERAL.groupby([VENTAS_GENERAL['fecha_creacion'].dt.to_period('M')])['total'].sum().reset_index()
monthly_user_sales

,fecha_creacion,total
0,2023-12,208.35


### PRODUCTOS VENDIDOS POR VENDEDOR

In [9]:
SQL_VENTAS = "select * from ventas v left join usuarios u on u.id_usuario = v.id_usuario_fk;"

In [20]:
with alchemyEngine.connect() as dbConnection:
  VENTAS_PRODUCTOS = pd.read_sql(
    "select pv.id_producto_fk, pv.cantidad, pv.total, u.name_user, v.fecha_creacion from productos_ventas pv left join productos p on p.id_producto = pv.id_producto_fk left join ventas v on pv.id_venta_fk  = v.id_venta left join usuarios u on v.id_usuario_fk = u.id_usuario",
    dbConnection,
  )

  ALL_PRODUCTS = pd.read_sql(
    "select id_producto, nombre, precio from productos p",
    dbConnection,
  )

In [21]:
VENTAS_PRODUCTOS

,id_producto_fk,cantidad,total,name_user,fecha_creacion
0,1,4,8.0,Ximena García,2024-01-15 18:43:08.322902


In [15]:
VENTAS_PRODUCTOS['fecha_creacion'] = pd.to_datetime(VENTAS_PRODUCTOS['fecha_creacion'])
VENTAS_PRODUCTOS


,id_producto_fk,cantidad,total,name_user,fecha_creacion
0,1,4,8.0,Ximena García,2024-01-15 18:34:18.249753+00:00


In [17]:
# Convierte la columna al formato de PostgreSQL sin zona horaria
VENTAS_PRODUCTOS['fecha_creacion'] = VENTAS_PRODUCTOS['fecha_creacion'].dt.tz_localize(None)
VENTAS_PRODUCTOS

,id_producto_fk,cantidad,total,name_user,fecha_creacion
0,1,4,8.0,Ximena García,2024-01-15 18:34:18.249753


In [14]:
VENTAS_PRODUCTOS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id_producto_fk  1 non-null      int64              
 1   cantidad        1 non-null      int64              
 2   total           1 non-null      float64            
 3   name_user       1 non-null      object             
 4   fecha_creacion  1 non-null      datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(1)
memory usage: 168.0+ bytes


In [44]:
VENTAS_PRODUCTOS

,id_producto_fk,cantidad,total,name_user,fecha_creacion
0,36,1,0.5,Ximena García,2023-12-29 16:43:55.314302
1,36,1,0.5,Alexandra Sinche,2023-12-29 21:27:04.593617
2,36,3,1.5,Alexandra Sinche,2023-12-29 21:27:04.593617
3,1,4,8.0,Ximena García,2024-01-06 14:44:23.058989


In [54]:
daily_user_sales = VENTAS_PRODUCTOS.groupby([VENTAS_PRODUCTOS['fecha_creacion'].dt.to_period('D'), 'name_user','id_producto_fk']).agg(
  {
    'cantidad': 'sum',
    'total': 'sum'}
  ).reset_index()
daily_user_sales
MERGE_VENTAS = pd.merge(daily_user_sales, ALL_PRODUCTS, left_on='id_producto_fk', right_on='id_producto', how='left')
MERGE_VENTAS = MERGE_VENTAS[[
  'fecha_creacion',
  'name_user',
  'nombre',
  'precio',
  'cantidad',
  'total'
  ]]
MERGE_VENTAS.rename(columns = {
  'fecha_creacion':'Fecha de Venta',
  'name_user':'Usuario',
  'nombre':'Producto',
  'precio':'Precio Unitario',
  'cantidad':'Cantidad',
  'total':'Total',
}, inplace = True)

MERGE_VENTAS

,Fecha de Venta,Usuario,Producto,Precio Unitario,Cantidad,Total
0,2023-12-29,Alexandra Sinche,AGUA,0.5,4,2.0
1,2023-12-29,Ximena García,AGUA,0.5,1,0.5
2,2024-01-06,Ximena García,SOLANO 2 ALITAS + PAPAS,2.0,4,8.0


In [41]:
VENTAS_PRODUCTOS_GROUPED = VENTAS_PRODUCTOS.groupby(['id_producto_fk', ]).agg(
  {
    'name_user': 'first',
    'cantidad': 'sum',
    'total': 'sum'}
  )#.reset_index()
VENTAS_PRODUCTOS_GROUPED
#VENTAS_PRODUCTOS_GROUPED.unstack(level=2)

,name_user,cantidad,total
id_producto_fk,,,
1,Ximena García,4,8.0
36,Ximena García,5,2.5


In [19]:
VENTAS_PRODUCTOS_GROUPED = VENTAS_PRODUCTOS_GROUPED.set_index('name_user', inplace=True)
VENTAS_PRODUCTOS_GROUPED

KeyError: "None of ['name_user'] are in the columns"

In [8]:
df_pivot = VENTAS_PRODUCTOS.pivot_table(index='name_user', columns=['id_producto_fk'], values=['cantidad', 'total'], aggfunc='sum')
df_pivot.columns = df_pivot.columns.map(lambda x: f'{x[0]}_{x[1]}')
df_pivot.reset_index(inplace=True)
df_pivot.columns = ['name_user'] + df_pivot.columns[1:].tolist()
df_pivot

,name_user,cantidad_1,cantidad_36,total_1,total_36
0,Alexandra Sinche,NaN,4.0,NaN,2.0
1,Ximena García,4.0,1.0,8.0,0.5
